# What is Escher?

Escher is a web-based tool for building, viewing, and sharing visualizations of biological pathways. These 'pathway maps' are a great way to contextualize data about metabolism, including the predictions made with COBRA and Cameo.

### Links

- Homepage: http://escher.github.io
- Documentation: https://escher.readthedocs.org
- Publication: https://doi.org/10.1371/journal.pcbi.1004321

# Try the Escher application (10 min)

Escher is a web application, and you can use it by visiting the [homepage](http://escher.github.io) from any browser.

Try it now. Visit the homepage and click Load Map to see an example of an interactive Escher map. Try:

- Load the map called "Central metabolism (iJO1366)" for _Escherichia coli_
- Zoom and pan on the map. Drag the map to pan. Use the + and - buttons on the left to zoom (or the + and - keys).
- Search by selecting **Find** from the **View** menu at the top. You can search by reaction, metabolite, or gene.
- Save the map as a JSON file in the **Map** menu. This file represents the current Escher map, and you can load it in later.
- Save the map as SVG in the **Map** menu. This saves an image of the map that is great for generating figures with application like Adobe Illustrator and Inkscape.

In Escher, you can also edit existing pathways, draw new pathways, and save the map in a number of formats. You will find more detail in the [docs](https://escher.readthedocs.org). Try some of these features out if you have time.

# Escher for Python (20 min)

We also provide a Python package to create Escher visualizations from Python code and embed them in Jupyter notebooks. This is especially useful if you are making predictions or analyzing data in Python, and you want a quick visualization.

In this notebook, we'll go through the key features of Escher that can be used to visualize predicted fluxes from COBRA.

To get started, import escher, cobra, and cameo.

In [ ]:
import escher
import cobra
import cameo

## Finding maps

The maps on the Escher website are also available from Python. You can list them like this.

In [ ]:
escher.list_available_maps()

## Launch the builder

Use the `Builder` class to create a new Escher map. You can pass a `map_name` from `list_available_maps` for a pre-built map. Then display it in the notebook.

This map will let you pan and zoom, but editing features are not enabled.

In [ ]:
b = escher.Builder(map_name='e_coli_core.Core metabolism')
b.display_in_notebook()

## Plot FBA solutions in Escher

Flux predictions are easy to plot on the map. Let's first generate a flux vector in COBRApy.

In [ ]:
model = cobra.io.read_sbml_model('data/e_coli_core.xml.gz')
solution = model.optimize()
print('Growth rate: %.2f' % solution.objective_value)

Now these we have a solution, let's visualize it on the map. To improve the visual style, you can adjust many options of the Builder. Some of the most useful ones are:

- reaction_scale: Pass a color & size scale as a list of points. You can set points for 'min', 'mean', 'max', 'median', 'Q1', 'Q3', and 'value'. These are the same options available in the Settings menu on the Escher website.
- hide_secondary_metabolite: Simplify the map by hiding cofactors.
- reaction_styles: Pass an array with any or 'size' to size reactions, 'color' to color them, 'text' to include data in their text labels, and 'abs' to visualize the absolute value of your data

Similar settings are available for metabolite data, and they are all described in the [docs](https://escher.readthedocs.io/en/stable/python_api.html).

In [ ]:
b = escher.Builder(map_name='e_coli_core.Core metabolism',
                   reaction_data=dict(solution.fluxes),
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'value', 'value': 0.1, 'color': '#cccccc', 'size': 8},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
                   # absolute value and no text for data
                   reaction_styles=['size', 'color', 'abs'],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
b.display_in_notebook()

# Escher for experimental data (20min)

Escher can be used to visualize any experimental data associated with reactions, metabolites, or genes.

Reaction-associated data is loaded just like fluxes. Each data point should have a key that is a reaction ID on the map and a floating point value.

## Metabolite data

For metabolite data, each point should have a key that is a metabolite ID on the map. Let's look at an existing CSV file of metabolomics data.

In [ ]:
import pandas as pd
metabolomics = pd.read_table('data/S4_McCloskey2013_aerobic_metabolomics.csv', sep=',', header=None)
metabolomics.head()

Escher expects a dictionary, so let's make a dictionary out of this data and pass it into a Builder as `metabolite_data`.

In [ ]:
metabolomics_dict = dict(metabolomics.values)

In [ ]:
b = escher.Builder(map_name='e_coli_core.Core metabolism',
                   metabolite_data=metabolomics_dict,
                   metabolite_scale=[
                       {'type': 'min', 'color': 'white', 'size': 10},
                       {'type': 'median', 'color': 'green', 'size': 20},
                       {'type': 'max', 'color': 'red', 'size': 40},
                   ],
                   enable_tooltips=False, 
                  )
b.display_in_notebook()

## Gene data & dataset comparison with Escher

For metabolite data, each point should have a key that is a gene ID on the map. To see the genes, first load a map with the option `show_gene_reaction_rules=True`.

Escher also allows you to load two datasets and visualize a comparison, so we will try that with gene data.

In [ ]:
b = escher.Builder(map_name='e_coli_core.Core metabolism',
                   show_gene_reaction_rules=True,
                  )
b.display_in_notebook()

You can provide genes by ID (the locus tags show on the map) or by name (you can see these when you hover over a gene).

Let's load some example RNA-seq data.

In [ ]:
rnaseq = pd.read_table('data/S6_RNA-seq_aerobic_to_anaerobic.csv', sep=',', header=0, index_col=0)
rnaseq.head()

Notice that we have two datasets here. For multiple datasets, Escher expects an array of data dictionaries. E.g.:

`[ { 'b0001': 4895.5133 ... }, { 'b0001':  8567.3833 ...} ]`

We can make that array like this:

In [ ]:
rnaseq_array = [dict(zip(rnaseq.index, x)) for x in rnaseq.values.T]

Let's plot it on the map! Genes are visualized on reactions, so the `reaction_scale` and `reaction_styles` options still work here. We also have new options for `reaction_compare_style` which can be 'fold', 'log2_fold', or 'diff' and define how the two datasets are compared on the map.

In [ ]:
b = escher.Builder(map_name='e_coli_core.Core metabolism',
                   gene_data=rnaseq_array,
                   reaction_compare_style='log2_fold',
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': 'green', 'size': 25},
                                   {'type': 'value', 'value': 0, 'color': '#cccccc', 'size': 8},
                                   {'type': 'max', 'color': 'red', 'size': 25}],
                   # absolute value and no text for data
                   reaction_styles=['size', 'color', 'text'],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
b.display_in_notebook()

Right away, we can see that genes activated in anaerobic conditions (e.g. fermentation pathways) are red, and genes activated in aerobic conditions (e.g. TCA cycle) are green.

# Editing maps and loading custom maps in Escher (10min)

Escher lets you build your own maps for any model. The Jupyter notebook is not a great environment for interactiving heavily with the maps, so editing is easier on the website. All you need is a JSON file for your COBRA model to serve as the source of content for your new map. You can also build new maps with the models already available on the Escher website, or you can edit existing maps to fit your needs.

There are some more tips on this page in the documentation: 

https://escher.readthedocs.io/en/stable/contribute_maps.html


Once you have a custom map, you can visualized it in the Escher Python package by passing a filename to `map_json`.

In [ ]:
# pass the model to a new builder
b = escher.Builder(map_json='data/custom_map.json')
b.display_in_notebook()

To make the file accessible to Escher, go to the Jupyter file manager and use the Upload button to add your custom map to the data folder. Then try visualizing it here in Escher:

In [ ]:
b = escher.Builder(map_json='data/CHANGE_ME.json')
b.display_in_notebook()